# Logistisches Modell

In [ ]:
import pathlib
import urllib

import matplotlib.pyplot as plt
import pandas as pd

In dieser Hausaufgabe sollen Sie die Kostenfunktion einer logistischen Regression visualisieren. Das passiert im anderen Notebook der Hausaufgabe. Dort benötigen Sie aber Daten, die Sie zunächst aufbereiten, kodieren, und visualisieren sollen. Dieser Teil geschieht in diesem Notebook.

## [5P] Daten einlesen und verarbeiten

Als Datengrundlage bietet sich der Iris-Datensatz an. Laden Sie ihn zunächst herunter, falls erforderlich, und lesen ihn ein:

In [ ]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
data_file = pathlib.Path(url.rsplit('/', 1)[-1])

if not data_file.is_file():
    # We could also use `df_iris = pd.read_csv(url)`
    # but here we save to disk to avoid unnecessary repeated downloads.
    urllib.request.urlretrieve(url, data_file)

df_iris = pd.read_csv(data_file.resolve(), header=None)

Für diese Aufgabe benötigen Sie einen eindimensionalen Datensatz, dem genau ein Label zugeordnet ist. In den Spalten sind die folgenden Größen in dieser Reihenfolge enthalten:

In [ ]:
column_names = ["sepal length", "sepal width", 'petal length', 'petal width', 'species']

**[1P]** Ordnen zunächst dem DataFrame die Spaltennamen und Datentypen zu.

In [ ]:
...

**[4P]** Für unser Modell beschränken wir uns auf die Kelchblattlänge (`'sepal length'`) der Spezies `Iris-virginica` und `Iris-setosa`. Stellen Sie für diese beiden Spezies die Verteilung der Kelchblattlängen grafisch in einem Diagramm dar.

In [ ]:
...

## [5P] Daten auswählen und Variablen kodieren

Als nächstes sollen Sie den Datensatz filtern und die Variablen kodieren. Beschränken Sie sich auf alle Datenpunkte der Kelchblattlänge, die zu einer der beiden obigen Spezies gehören. Setzen Sie den Index zurück. Ersetzen Sie die Spalte `species` durch eine Series mit Einträgen vom Typ Integer, und zwar `0`, falls es sich um `Iris-setosa` handelt, und `1`, falls es sich um `Iris-virginica` handelt.

Denken Sie daran, idiomatischen, verständlichen, und performanten Code zu schreiben — auch wenn dieser Datensatz noch klein ist.

In [ ]:
...

## Logistisches Modell

Um für eine kontinuierliche Variable $x$ ein binäres Label $y$ vorherzusagen, bietet sich ein logistisches Modell an. Hier ist $x$ die Kelchblattlänge und $y$ die Spezies der zugehörigen Pflanze. Die Daten wurden so kodiert, dass $y=0$ bzw. $y=1$ der Spezies "Iris setosa" bzw. "Iris virginica" entspricht. Das haben Sie in der vorherigen Teilaufgabe getan — dieser Zwischenstand ist allerdings auch mitgeliefert.

Die Vorhersage eines logistischen Modells $\hat{y}(x)$ ist dann die *Wahrscheinlichkeit*, mit der es sich bei einer Pflanze dieser Kelchblattlänge um die Spezies "Iris virginica" handelt. Für eine Variable hat es zwei freie Parameter $a_0$ und $a_1$ und lautet:

$$\hat{y}(x) = \frac{1}{1 + \mathrm{e}^{- (a_0 + a_1 x)}}$$

In [ ]:
def logistic_model(x, a_0: float, a_1: float) -> float:
    """Logistic model for one continuous variable.

    a_0, a_1 -- free parameters of the logistic model.
    x -- continuous scalar variable

    Returns a probability 0 < p < 1 under a logistic model of `a_0 + a_1 x`.
    """
    return 1 / (1 + np.exp(-(a_0 + a_1 * x)))

## [3P] Daten einlesen und visualieren

Lesen Sie nun den fertig kodierten Datensatz ein:

In [ ]:
coded_df_iris = pd.read_csv('coded_df_iris.csv')

Stellen Sie in einer gemeinsamen Abbildung die Datenpunkte sowie das logistische Modell mit $a_0 = -80;~~a_1 = 15$ sinnvoll im gesamten Datenbereich dar.

In [ ]:
...

## [3P] Wert der Kostenfunktion für ausgewählte Modelle

Als Kostenfunktion für logistische Regression dient die "mittlere Kreuzentropie":

In [ ]:
def average_cross_entropy(predictions, truth) -> float:
    """Calculate average cross entropy between two arrays.

    Typically, `predictions` would hold predicted probabilities for labels
    and `truth` would hold the observed values.

    But really any two Series with values between 0 and 1 work.
    """
    cross_entropies = -np.log(truth * predictions + (1 - truth) * (1 - predictions))
    return cross_entropies.mean()

Berechnen Sie die mittlere Kreuzentropie zwischen den Daten und dem logistischen Modell mit $a_0 = -80; ~~ a_1 = 15$ sowie vier weiteren logistischen Modellen mit von Ihnen gewählten Werten für $a_0$ und $a_1$.

Stellen Sie diese Modelle gemeinsam mit den Daten und den mittleren Kreuzentropien sinnvoll gemeinsam dar.

In [ ]:
...

## [3P] Konturplot der Kostenfunktion

Erstellen Sie einen Konturplot der Kostenfunktion. Tragen Sie dazu die mittlere Kreuzentropie eines logistischen Modells mit den Parametern $a_0$ und $a_1$ als Konturplot gegen die Parameter $a_0$ und $a_1$ auf. Wählen Sie einen sinnvollen Wertebereich für die Parameter. Welche Form hat der Bereich mit "kleinen" Werten der Kostenfunktion? Wann und warum treten sehr große Werte auf?

Hilfe zur Erstellung eines Konturplots mit matplotlib finden Sie z.B. unter https://alex.miller.im/posts/contour-plots-in-python-matplotlib-x-y-z/. Es bietet sich an, zunächst die `np.meshgrid`s für `a_0` und `a_1` zu erstellen, die Daten dann in das auf dieser Seite beschriebene "long"-Format umzuwandeln, dort die Kostenfunktion auszuwerten, und dann wie dort beschrieben mit der `pivot_table`-Methode die Werte der Kostenfunktion in das zum Plotten benötigte Format umzuwandeln.

In [ ]:
...

## [1P] Mit den Daten herumspielen

Sobald Sie für einen Datensatz die Kostenfunktion mit einem Konturplot visualisiert haben, sollte es relativ leicht sein, dies für andere Datensätze zu wiederholen. In der csv-Datei sind auch andere Größen und alle Spezies kodiert.

Wählen Sie z.B. eine andere Messgröße, eine andere Kombination von Spezies (achten Sie dann auf deren Kodierung!), oder transformieren Sie Ihre Daten vor Darstellung der Kostenfunktion. In der Vorlesung wurde beispielsweise Skalierung von Mittelwert und Standardabweichung als sinnvolle Transformation oder die Kostenfunktion "Mean Squared Error" als nicht für logistische Regression geeignet angesprochen. Sie könnten auch dies implementieren und diskutieren.

Probieren Sie mehr als einen Ansatz aus und setzen Sie diese um. Es ist nicht so wichtig, *was* Sie genau tun, wie *dass* Sie mehrere Dinge versuchen und zumindest ein interessantes Ergebnis erhalten und sinnvoll visualisieren. Dokumentieren Sie Ihr Vorgehen sinnvoll.

In [ ]:
...